In [ ]:
#!pip install langchain-community
#!pip install langchain-text-splitters
#!pip install langchain
#!pip install python-dotenv
#!pip install pytube
#!pip install langchain-openai
#!pip install youtube-transcript-api
#!pip install chromadb
#!pip install gradio

In [1]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.chains.question_answering import load_qa_chain
import gradio as gr
from youtube_transcript_api import NoTranscriptFound
from dotenv import load_dotenv
load_dotenv()
import os

with open("video_link.txt", "r") as file:
    urls = file.readlines()
    urls = [url.strip() for url in urls]

all_data = []

for url in urls:
    try:
        loader = YoutubeLoader.from_youtube_url(
            url, add_video_info=False, language="tr", translation="en"
        )
        text = loader.load_and_split()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=0)
        data = text_splitter.split_documents(text)

        for document in data:
            document.page_content = f"{url}, {document.page_content}"

        all_data.extend(data)
    except NoTranscriptFound:
        print(f"No transcript : {url}. skipping...")

c:\Users\quari\miniconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = OpenAI(temperature=1, openai_api_key=os.environ.get("OPENAI_API_KEY"))
embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

db = Chroma.from_documents(all_data, embedding=embeddings, persist_directory="test_index")
db.persist()

c:\Users\quari\miniconda3\envs\nlp\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [1]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.chains.question_answering import load_qa_chain
import gradio as gr
from youtube_transcript_api import NoTranscriptFound
from dotenv import load_dotenv
load_dotenv()
import os

c:\Users\quari\miniconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = OpenAI(temperature=1, openai_api_key=os.environ.get("OPENAI_API_KEY"))
embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
vectordb = Chroma(persist_directory="test_index", embedding_function=embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

chain = load_qa_chain(llm, chain_type="stuff")

def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = chain({"input_documents": context, "question": question}, return_only_outputs=True)['output_text']

    video_url = context[0].page_content.split(',')[0]

    return answer, video_url

def get_recipe(question):
    answer, video_url = ask(question)
    video_iframe = f'<iframe width="560" height="315" src="{video_url.replace("watch?v=", "embed/")}" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
    return answer, video_iframe

iface = gr.Interface(
    fn=get_recipe,
    inputs=gr.components.Textbox(lines=2, label="Ask a question about a recipe..."),
    outputs=[
        gr.components.Textbox(label="Recipe"),
        gr.components.HTML(label="YouTube Video"),
    ],
    title="Ne Yesem?",
    description="Ask a question about a recipe and get the answer along with a related YouTube video.",
)

iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://db6a973b73b4294c44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


c:\Users\quari\miniconda3\envs\nlp\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
c:\Users\quari\miniconda3\envs\nlp\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
